Print pandas version. IF Pandas Version > 1.0 then 

* from pandas.io.json import json_normalize -> from pandas import json_normalize

In [1]:
import sys
import pandas
print("Python Version : ", sys.version)
print("Pandas Version : ", pandas.__version__)

Python Version :  3.8.6 | packaged by conda-forge | (default, Oct  7 2020, 19:08:05) 
[GCC 7.5.0]
Pandas Version :  1.1.3


Embedding ES query : https://www.tutorialspoint.com/elasticsearch/elasticsearch_query_dsl.htm

In [ ]:
import requests, json

import pandas as pd
from pandas.io.json import json_normalize

class KibanaClient:
    '''This is the Kibana Client.'''
    def __init__(self, hostname, port=5601, protocol='http'):
        self.base_url = "{2}://{0}:{1}".format(hostname, port, protocol)
        # https://www.elastic.co/guide/en/kibana/master/using-api.html#api-request-headers
        self.headers = {'kbn-xsrf': 'true', 'Accept': 'application/json', 'Content-Type': 'application/json'}

    def dashboard(self, dashboard, debug=False):
        '''Read JSON from URL'''
        url = self.base_url + '/api/kibana/dashboards/export?dashboard={0}'.format(dashboard)
        if (debug):
            print("Read url from :", url)
        content = requests.get(url, headers=self.headers, allow_redirects=True, verify=False)
        json_obj = json.loads(content.content)
        return json_obj        

    def readJsonURL(self, path, body="", debug=False):
        '''Read JSON from URL'''
        url = self.base_url + path
        if (debug):
            print("Read url from :", url)
        content = requests.post(url, headers=self.headers, data=body, allow_redirects=True)
        json_obj = json.loads(content.content)
        return json_obj
    
    def indices(self, debug=False):
        json_obj = self.readProxyURL("GET", "/_cat/indices?format=json", debug)
        return json_obj

    def indicesDF(self, debug=False):
        json_obj = self.indices(debug)
        df = pd.DataFrame.from_dict(json_obj, orient='columns')
        return df

    def searchRangeQueryDF(self, index, field, operator, value, debug=False):
        json_obj = self.searchRangeQuery(index, field, operator, value, debug)
        return self.resultToDF(json_obj);
        
    def searchRangeQuery(self, index, field, operator, value, debug=False):
        query = {
            field: {
                operator: value
            }
        }
        return self.searchRange(index, query, debug)
        
    def searchTextDF(self, index, text, debug=False):
        json_obj = self.searchText(index, text, debug)
        return self.resultToDF(json_obj);
        
    def searchText(self, index, text, debug=False):
        query = {
            'query_string': {
                'query': text
            }
        }
        return self.search(index, query, debug)

    def searchRange(self, index, range_query, debug=False):
        query = {
            'range': range_query
        }
        return self.search(index, query, debug)

    def searchTermsDF(self, index, terms, debug=False):
        json_obj = self.searchTerms(index, terms, debug)
        return self.resultToDF(json_obj);
    
    def searchTerms(self, index, terms, debug=False):
        query = {
            'term': terms
        }
        return self.search(index, query, debug)

    def searchMatchDF(self, index, matches, debug=False):
        json_obj = self.searchMatch(index, matches, debug)
        return self.resultToDF(json_obj);

    def searchMatch(self, index, matches, debug=False):
        query = {
            'match': matches
        }
        return self.search(index, query, debug)
    
    def searchDF(self, index, query, debug=False):
        json_obj = self.search(index, query, debug)
        return self.resultToDF(json_obj);
    
    def search(self, index, query, debug=False):
        path = "/{0}/_search".format(index)
        query_dict = {
            'query': query
        }
        if (debug):
            print("Query :", query_dict)
        query_json = json.dumps(query_dict)
        json_obj = self.readProxyURL("POST", path, query_json, debug)       
        if (debug):
            print("Found result :", json_obj)
        return json_obj
    
    def resultToDF(self, json_obj):
        df = pd.DataFrame.from_dict(json_obj['hits']['hits'], orient='columns')
        return df        

    def readProxyURL(self, method, path, body="", debug=False):
        return self.readJsonURL('/api/console/proxy?method={0}&path={1}'.format(method, path), body, debug)


In [ ]:
from pyspark.sql import SparkSession

class KibanaPySpark:
    '''This is the Kibana PySpark.'''
    def __init__(self, hostname, port=5601, protocol='http'):
        self.client = KibanaClient(hostname, port, protocol)

        self.spark = SparkSession.builder.appName('pyspark-' + hostname).getOrCreate()
        #set timestamp
        self.spark.sql("set spark.sql.session.timeZone=UTC")

    def indicesDF(self, debug=False):
        df = self.client.indicesDF(debug)
        return self.spark.createDataFrame(df)

    def searchRangeQueryDF(self, index, field, operator, value, debug=False):
        df = self.client.searchRangeQueryDF(index, field, operator, value, debug)
        return self.spark.createDataFrame(df)
 
    def searchTextDF(self, index, text, debug=False):
        df = self.client.searchTextDF(index, text, debug)
        return self.spark.createDataFrame(df)
        
    def searchTermsDF(self, index, terms, debug=False):
        df = self.client.searchTermsDF(index, terms, debug)
        return self.spark.createDataFrame(df)
    
    def searchMatchDF(self, index, matches, debug=False):
        df = self.client.searchMatchDF(index, matches, debug)
        return self.spark.createDataFrame(df)

    def searchDF(self, index, query, debug=False):
        df = self.client.searchDF(index, query, debug)
        return self.spark.createDataFrame(df)
